# Setup

In [15]:
# import packages and modules
import os
import csv
from datetime import datetime
import pyodbc
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pickle
import pandas as pd
from functools import reduce

# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "in_memory" + "\\"

# set workspace and sde connections 
scratchFolder = "C:\\GIS\\PROJECT\\ScratchPaper"
workspace     = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb"
arcpy.env.workspace = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb"

## SDE Connection Files saved on the Network
# sdeTabular = "F:\\GIS\\GIS_DATA\\Tabular.sde"
# sdeBase    = "F:\\GIS\\GIS_DATA\\Vector.sde"
# sdeCollect = "F:\\GIS\\GIS_DATA\\Collect.sde"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

In [16]:
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
   
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
   
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

## Get a Copy of Parcel Master

In [9]:
# copy parcel master to local gdb
# parcel base
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# copy
arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelMaster")

<Result 'C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\ParcelMaster'>

In [27]:
# create spatial dataframe from parcel feature class
sdfParcels = pd.DataFrame.spatial.from_featureclass('ParcelMaster')
sdfParcels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61263 entries, 0 to 61262
Data columns (total 62 columns):
OBJECTID                          61263 non-null int64
APN                               61263 non-null object
PPNO                              61263 non-null float64
HSE_NUMBR                         61263 non-null int64
UNIT_NUMBR                        61263 non-null object
STR_DIR                           61263 non-null object
STR_NAME                          61263 non-null object
STR_SUFFIX                        61263 non-null object
APO_ADDRESS                       61263 non-null object
PSTL_TOWN                         61263 non-null object
PSTL_STATE                        61263 non-null object
PSTL_ZIP5                         61263 non-null object
OWN_FIRST                         61263 non-null object
OWN_LAST                          61263 non-null object
OWN_FULL                          61263 non-null object
MAIL_ADD1                         61263 non-null obj

## Get LTInfo Data Frames

In [4]:
# create dataframes from https://qa.laketahoeinfo.org/WebServices/List
dfLTAPN    = pd.read_json("https://laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV      = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDevBank  = pd.read_json("https://laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

### Dump Pickles

In [11]:
# dump pickle of merged dataframe
with open(os.path.join(scratchFolder, "sdfParcels.pickle"), 'wb') as f:
    pickle.dump(sdfParcels, f)
    
# dump pickle of merged dataframe
with open(os.path.join(scratchFolder, "dfLTAPN.pickle"), 'wb') as f:
    pickle.dump(dfLTAPN, f)

# dump pickle of merged dataframe
with open(os.path.join(scratchFolder, "dfIPES.pickle"), 'wb') as f:
    pickle.dump(dfIPES, f)

# dump pickle of merged dataframe
with open(os.path.join(scratchFolder, "dfLCV.pickle"), 'wb') as f:
    pickle.dump(dfLCV, f)
    
# dump pickle of merged dataframe
with open(os.path.join(scratchFolder, "dfDevBank.pickle"), 'wb') as f:
    pickle.dump(dfDevBank, f)

### Load Pickles

In [13]:
with open(os.path.join(scratchFolder, "sdfParcels.pickle"), 'rb') as f:
    sdfParcels = pickle.load(f)

with open(os.path.join(scratchFolder, "dfLTAPN.pickle"), 'rb') as f:
    dfLTAPN = pickle.load(f)

with open(os.path.join(scratchFolder, "dfIPES.pickle"), 'rb') as f:
    dfIPES = pickle.load(f)
    
with open(os.path.join(scratchFolder, "dfLCV.pickle"), 'rb') as f:
    dfLCV = pickle.load(f)

with open(os.path.join(scratchFolder, "dfDevBank.pickle"), 'rb') as f:
    dfDevBank = pickle.load(f)

# Accela to SDE

## BMP Status Update

In [28]:
sdfParcels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61263 entries, 0 to 61262
Data columns (total 62 columns):
OBJECTID                          61263 non-null int64
APN                               61263 non-null object
PPNO                              61263 non-null float64
HSE_NUMBR                         61263 non-null int64
UNIT_NUMBR                        61263 non-null object
STR_DIR                           61263 non-null object
STR_NAME                          61263 non-null object
STR_SUFFIX                        61263 non-null object
APO_ADDRESS                       61263 non-null object
PSTL_TOWN                         61263 non-null object
PSTL_STATE                        61263 non-null object
PSTL_ZIP5                         61263 non-null object
OWN_FIRST                         61263 non-null object
OWN_LAST                          61263 non-null object
OWN_FULL                          61263 non-null object
MAIL_ADD1                         61263 non-null obj

In [35]:
# start timer
startTimer = datetime.now()

# name of feature class
outFC = "Parcel_BMP"

# make sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM tahoebmpsde.dbo.v_BMPStatus", conn)

# specify sde connection file
sdeBase = "C:\\GIS\\DB_CONNECT\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, on='APN', how='inner')

# specify fields to keep
dfOut = df[['APN',
            'JURISDICTION',
            'ZONING_ID',
            'PLAN_NAME',
            'TOWN_CENTER',
            'CertificateIssued',
            'EvaluationComplete',
            'SourceCertIssued',
            'CertDate',
            'CertReissuedDate',
            'LandUse',
            'BMPStatus',
            'Catchment',
            'SourceCertDate',
            'SiteConstraint',
            'CreditPercent',
            'AreaWide',
            'CreditArea',
            'TMDL_LandUse',
            'CertNo',
            'SHAPE'
            ]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))




Updated Parcel_BMP

Time it took to run this script: 0:03:28.996381


In [37]:
# start timer
startTimer = datetime.now()

## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = "Parcel_BMP"

# Change this to the path of your output FC
outfc = sdeCollect + "\\sde_collection.SDE.Parcel\\sde_collection.SDE.Parcel_BMP"

# feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"

fieldnames =  ['APN',
            'JURISDICTION',
            'ZONING_ID',
            'PLAN_NAME',
            'TOWN_CENTER',
            'CertificateIssued',
            'EvaluationComplete',
            'SourceCertIssued',
            'CertDate',
            'CertReissuedDate',
            'LandUse',
            'BMPStatus',
            'Catchment',
            'SourceCertDate',
            'SiteConstraint',
            'CreditPercent',
            'AreaWide',
            'CreditArea',
            'TMDL_LandUse',
            'CertNo',
            'SHAPE@']

#--------------------------------------------------------------------------------------------------------#

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print ("Unregistering feature dataset as versioned...")
# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("Finished unregistering feature dataset as versioned.")
# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print("\nRegistering feature dataset as versioned...")
# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
print("\nFinished registering feature dataset as versioned.")

# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))


Disconnecting all users...
Unregistering feature dataset as versioned...
Finished unregistering feature dataset as versioned.


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
The featureclass was not found.
Failed to execute (TruncateTable).


## LCV Update

In [ ]:

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_LCV"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name


# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_LandCapabilityVerifications", conn)

# specify sde connection file
sdeBase = "C:\\GIS\\DB_CONNECT\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)


## LCV Challenge Update

In [ ]:

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_LCV_Challenge"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name

# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_LandCapabilityChallenges", conn)

# specify sde connection file
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"REC_DATE": "Date", "LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Date", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

## Soils Hydro Update

In [ ]:

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_SoilsHydro"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name

# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_HydroSoilsProjects", conn)

# specify sde connection file
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"REC_DATE": "Date", "LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Date", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

## Production Script

In [ ]:
"""
ETL_Accella_To_SDE.py
March 13th, 2020
Mason Bindl, Tahoe Regional Planning Agency

This python script was developed to move data from 
Accela to TRPA's Enterprise Geodatabase, Collection.sde

This script uses Python 3.x and was designed to be used with 
the default ArcGIS Pro python enivorment "arcgispro-py3", with
no need for installing new libraries.

"""

## This script updates parcel specific feature classes for BMPs, LCVs, LCCs, Historic Parcels,
# Import system modules
import pandas as pd
import pyodbc
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from datetime import datetime

# set workspace and sde connections
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_BMP"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name

# make sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM tahoebmpsde.dbo.v_BMPStatus", conn)

# specify sde connection file
sdeBase = "C:\\GIS\\DB_CONNECT\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, on='APN', how='inner')

# specify fields to keep
dfOut = df[['APN',
            'JURISDICTION',
            'CertificateIssued',
            'EvaluationComplete',
            'SourceCertIssued',
            'CertDate',
            'CertReissuedDate',
            'LandUse',
            'BMPStatus',
            'Catchment',
            'SourceCertDate',
            'SiteConstraint',
            'CreditPercent',
            'AreaWide',
            'CreditArea',
            'TMDL_LandUse',
            'CertNo',
            'SHAPE'
            ]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

# -------------------------------------------------------------- #

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_LCV"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name


# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_LandCapabilityVerifications", conn)

# specify sde connection file
sdeBase = "C:\\GIS\\DB_CONNECT\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

# -------------------------------------------------------------- #

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_LCV_Challenge"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name

# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_LandCapabilityChallenges", conn)

# specify sde connection file
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"REC_DATE": "Date", "LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Date", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

# --------------------------------------------------------------- #

# start timer
startTimer = datetime.now()

# name of feature class
name = "Parcel_SoilsHydro"
# specify output feature class
outFC = '//Arcprod/C$/GISData/PermitReview_AppData.gdb/' + name
# outFC = "C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\" + name

# make sql database connection with pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=ASQL;DATABASE=Accela;UID=BMP_Update;PWD=BMP_update_123')

# create dataframe from sql query
dfSQL = pd.read_sql("SELECT * FROM Accela.dbo.v_HydroSoilsProjects", conn)

# specify sde connection file
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

# create spatial dataframe from parcels in sde
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfSQL, left_on='APN', right_on='GIS_ID', how='inner')

# rename some of the fields
df.rename(columns={"REC_DATE": "Date", "LABEL_FIELD": "Status"}, inplace=True)

# specify fields to keep
dfOut = df[["APN", "Date", "Status", "SHAPE"]].copy()

# delete output fc if exists
if arcpy.Exists(outFC):
    arcpy.Delete_management(outFC)

# spaital dataframe to feature class
dfOut.spatial.to_featureclass(outFC)

# confirm feature class was created
print("\nUpdated " + outFC)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print("\nTime it took to run this script: {}".format(endTimer))

# SDE to SDE

## Parcel Edit to Parcel Master Staging

In [ ]:
# import modules
import arcpy
import os

# overwrite outputs to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
landuseFD = "sde_collection.SDE.LandUse"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, landuseFD)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")
try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


## Parcel Update to Parcel Edit

In [ ]:
# import modules
import arcpy
import os

# overwrite outputs to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
landuseFD = "sde_collection.SDE.LandUse"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, landuseFD)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")
try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


## Parcel Edit to Parcel Master

### Move Recent Land Use Edits to TRPA Land Use Field in Parcel Master

In [ ]:
# import modules
import arcpy
import os

# overwrite outputs to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
landuseFD = "sde_collection.SDE.LandUse"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, landuseFD)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")
try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


## BMP Database to Parcel Edit

### BMP Updated Land Use values to Parcel Edit Land Use

In [ ]:

# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
fd = "sde_collection.SDE.Parcel"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, fd)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")

try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


## Production Script

In [ ]:
"""
BMP_LandUse_To_ParcelCollect.py
March 13th, 2020
Mason Bindl, Tahoe Regional Planning Agency

This python script was developed to extract verified land use
values from the BMP database, transfer them to the Parcel_Collect
feature class in Collection SDE, then transfer all the land use values
from that dataset to Parcel_Master in Production SDE.

This script uses Python 3.x and was designed to be used with 
the default ArcGIS Pro python enivorment "arcgispro-py3", with
no need for installing new libraries.

"""
# import modules
import arcpy
import os

# overwrite outputs to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
fd = "sde_collection.SDE.Parcel"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, fd)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")

try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


# LTinfo to SDE

## LTInfo

In [46]:
# filter for active parcels
dfLTInfo_merge = dfLTAPN[(dfLTAPN.Status == "Active")]

# list of data frames to merge
data_frames = [dfLTInfo_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'],
                                            how='left'), data_frames)
# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_LTInfo")

'C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\Parcel_LTInfo'

In [ ]:
# start timer
startTimer = datetime.now()

# # copy parcel master to local gdb
# # parcel base
# parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelsBase")

# # create spatial dataframe from parcel feature class
# sdfParcels = pd.DataFrame.spatial.from_featureclass('ParcelsBase')
# sdfParcels.info()


# # load pickle from dataframe
# with open(os.path.join(dfDump , "dfIPES.pickle"), 'rb') as f:
#     dfIPES = pickle.load(f)

dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

# print data frame 
dfIPES.info()

# filter for active and official
dfIPES_merge = dfIPES[(dfIPES.Status == "Active") 
                            & (dfIPES.IPESScoreType == "Official")]

data_frames = [dfIPES_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'], how='left'), data_frames)

# print dataframe info
df.info()

# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_IPES")

# input fc
inputFC = "Parcel_IPES"

print("Imported arcpy, os, csv, and datatime modules.\n")
print(arcpy.ProductInfo())
## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = "Parcel_IPES"

# Change this to the path of your output FC
outfc = sdeCollect + "\\sde_collection.SDE.Parcel\\sde_collection.SDE.Parcel_LTinfo_IPES"

# feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"

fieldnames =  ['APN','PPNO','Status','ParcelNickname','IPESScore','IPESScoreType',
               'BaseAllowableCoveragePercent', 'IPESTotalAllowableCoverageSqFt',
               'ParcelHasDOAC', 'HistoricOrImportedIpesScore', 'CalculationDate',
               'FieldEvaluationDate', 'RelativeErosionHazardScore', 'RunoffPotentialScore',
               'AccessScore', 'UtilityInSEZScore', 'ConditionOfWatershedScore',
               'AbilityToRevegetateScore', 'WaterQualityImprovementsScore',
               'ProximityToLakeScore', 'LimitedIncentivePoints','TotalParcelArea',
               'IPESBuildingSiteArea', 'SEZLandArea','SEZSetbackArea','InternalNotes',
               'PublicNotes','PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE@']

#--------------------------------------------------------------------------------------------------------#

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print ("Unregistering feature dataset as versioned...")
# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("Finished unregistering feature dataset as versioned.")
# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print("\nRegistering feature dataset as versioned...")
# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
print("\nFinished registering feature dataset as versioned.")

# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))

## LCV

In [43]:
dfLCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21999 entries, 0 to 21998
Data columns (total 30 columns):
APN                    21999 non-null object
Status                 21999 non-null object
ParcelNickname         158 non-null object
TotalAreaSqFt          21419 non-null float64
UpdatedBy              21999 non-null object
UpdatedOn              21999 non-null object
DeterminationDate      21999 non-null object
EstimatedOrVerified    21999 non-null object
SitePlanUrl            21999 non-null object
AccelaCAPRecord        5906 non-null object
Bailey1aPresent        21999 non-null object
Bailey1aSqFt           510 non-null float64
Bailey1bPresent        21999 non-null object
Bailey1bSqFt           970 non-null float64
Bailey1cPresent        21999 non-null object
Bailey1cSqFt           51 non-null float64
Bailey2Present         21999 non-null object
Bailey2SqFt            267 non-null float64
Bailey3Present         21999 non-null object
Bailey3SqFt            278 non-null float64

In [49]:
# select fields
dfCoverageAllowed = dfLCV[['APN','Bailey1aSqFt', 'Bailey1bSqFt', 'Bailey1cSqFt','Bailey2SqFt','Bailey3SqFt','Bailey4SqFt', 'Bailey5SqFt', 'Bailey6SqFt', 'Bailey7SqFt','TotalAreaSqFt', 'Status', 'EstimatedOrVerified']]

# add coverage allowed field
dfCoverageAllowed["Developable_Area"] = np.nan

# calculate coverage allowed
dfCoverageAllowed['Developable_Area'] = dfCoverageAllowed[dfCoverageAllowed.columns[:-4]].sum(axis=1)

LCVquery = dfCoverageAllowed[(dfCoverageAllowed.Status == "Active") 
                            & (dfCoverageAllowed.EstimatedOrVerified == "Verified") 
                            & (dfCoverageAllowed.Developable_Area > 0)]

dfLCV_merge = LCVquery[['APN', 
                      'Developable_Area',
                      'TotalAreaSqFt', 
                      'Status', 
                      'EstimatedOrVerified']]

# list of dataframes to merge
data_frames = [dfLCV_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'],
                                            how='left'), data_frames)
# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_LCV")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\Parcel_LCV'

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3109 entries, 0 to 3108
Data columns (total 11 columns):
APN                    3109 non-null object
Developable_Area       3109 non-null float64
TotalAreaSqFt          3109 non-null float64
Status                 3109 non-null object
EstimatedOrVerified    3109 non-null object
OBJECTID               3109 non-null int64
PPNO                   3109 non-null float64
PARCEL_ACRES           3109 non-null float64
PARCEL_SQFT            3109 non-null float64
JURISDICTION           3109 non-null object
SHAPE                  3109 non-null geometry
dtypes: float64(5), geometry(1), int64(1), object(4)
memory usage: 267.3+ KB


In [ ]:
# start timer
startTimer = datetime.now()

# # copy parcel master to local gdb
# # parcel base
# parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelsBase")

# # create spatial dataframe from parcel feature class
# sdfParcels = pd.DataFrame.spatial.from_featureclass('ParcelsBase')
# sdfParcels.info()


# # load pickle from dataframe
# with open(os.path.join(dfDump , "dfIPES.pickle"), 'rb') as f:
#     dfIPES = pickle.load(f)

dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

# print data frame 
dfIPES.info()

# filter for active and official
dfIPES_merge = dfIPES[(dfIPES.Status == "Active") 
                            & (dfIPES.IPESScoreType == "Official")]

data_frames = [dfIPES_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'], how='left'), data_frames)

# print dataframe info
df.info()

# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_IPES")

# input fc
inputFC = "Parcel_IPES"

print("Imported arcpy, os, csv, and datatime modules.\n")
print(arcpy.ProductInfo())
## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = "Parcel_IPES"

# Change this to the path of your output FC
outfc = sdeCollect + "\\sde_collection.SDE.Parcel\\sde_collection.SDE.Parcel_LTinfo_IPES"

# feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"

fieldnames =  ['APN','PPNO','Status','ParcelNickname','IPESScore','IPESScoreType',
               'BaseAllowableCoveragePercent', 'IPESTotalAllowableCoverageSqFt',
               'ParcelHasDOAC', 'HistoricOrImportedIpesScore', 'CalculationDate',
               'FieldEvaluationDate', 'RelativeErosionHazardScore', 'RunoffPotentialScore',
               'AccessScore', 'UtilityInSEZScore', 'ConditionOfWatershedScore',
               'AbilityToRevegetateScore', 'WaterQualityImprovementsScore',
               'ProximityToLakeScore', 'LimitedIncentivePoints','TotalParcelArea',
               'IPESBuildingSiteArea', 'SEZLandArea','SEZSetbackArea','InternalNotes',
               'PublicNotes','PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE@']

#--------------------------------------------------------------------------------------------------------#

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print ("Unregistering feature dataset as versioned...")
# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("Finished unregistering feature dataset as versioned.")
# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print("\nRegistering feature dataset as versioned...")
# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
print("\nFinished registering feature dataset as versioned.")

# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))

## IPES

In [43]:
# start timer
startTimer = datetime.now()

# # copy parcel master to local gdb
# # parcel base
# parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelsBase")

# # create spatial dataframe from parcel feature class
# sdfParcels = pd.DataFrame.spatial.from_featureclass('ParcelsBase')
# sdfParcels.info()


# # load pickle from dataframe
# with open(os.path.join(dfDump , "dfIPES.pickle"), 'rb') as f:
#     dfIPES = pickle.load(f)

dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

# print data frame 
dfIPES.info()

# filter for active and official
dfIPES_merge = dfIPES[(dfIPES.Status == "Active") 
                            & (dfIPES.IPESScoreType == "Official")]

data_frames = [dfIPES_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'], how='left'), data_frames)

# print dataframe info
df.info()

# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_IPES")

print("Imported arcpy, os, csv, and datatime modules.\n")
print(arcpy.ProductInfo())
## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = "Parcel_IPES"

# Change this to the path of your output FC
outfc = sdeCollect + "\\sde_collection.SDE.Parcel\\sde_collection.SDE.Parcel_LTinfo_IPES"

# feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"

fieldnames =  ['APN','PPNO','Status','ParcelNickname','IPESScore','IPESScoreType',
               'BaseAllowableCoveragePercent', 'IPESTotalAllowableCoverageSqFt',
               'ParcelHasDOAC', 'HistoricOrImportedIpesScore', 'CalculationDate',
               'FieldEvaluationDate', 'RelativeErosionHazardScore', 'RunoffPotentialScore',
               'AccessScore', 'UtilityInSEZScore', 'ConditionOfWatershedScore',
               'AbilityToRevegetateScore', 'WaterQualityImprovementsScore',
               'ProximityToLakeScore', 'LimitedIncentivePoints','TotalParcelArea',
               'IPESBuildingSiteArea', 'SEZLandArea','SEZSetbackArea','InternalNotes',
               'PublicNotes','PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE@']

#--------------------------------------------------------------------------------------------------------#

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print ("Unregistering feature dataset as versioned...")
# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("Finished unregistering feature dataset as versioned.")
# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print("\nRegistering feature dataset as versioned...")
# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
print("\nFinished registering feature dataset as versioned.")

# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14651 entries, 0 to 14650
Data columns (total 27 columns):
ScoreSheetUrl                     14651 non-null object
APN                               14651 non-null object
Status                            14651 non-null object
ParcelNickname                    14651 non-null object
IPESScore                         14649 non-null float64
IPESScoreType                     14651 non-null object
BaseAllowableCoveragePercent      14651 non-null object
IPESTotalAllowableCoverageSqFt    861 non-null float64
ParcelHasDOAC                     14651 non-null object
HistoricOrImportedIpesScore       14651 non-null object
CalculationDate                   14651 non-null object
FieldEvaluationDate               14651 non-null object
RelativeErosionHazardScore        14470 non-null float64
RunoffPotentialScore              14470 non-null float64
AccessScore                       14470 non-null float64
UtilityInSEZScore                 14470 non-null

## Development Right

In [54]:
# list of dataframes to merge
data_frames = [dfDevBank, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'],
                                            how='left'), data_frames)
# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_DevelopmentRight")

'C:\\GIS\\PROJECT\\ScratchPaper\\ScratchPaper.gdb\\Parcel_DevelopmentRight'

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 17 columns):
APN                     1593 non-null object
DevelopmentRight        1593 non-null object
LandCapability          1593 non-null object
IPESScore               401 non-null float64
BankedQuanitity         1593 non-null int64
CurrentInventory        1593 non-null int64
Jurisdiction            1593 non-null object
LocalPlan               1573 non-null object
DateBankedOrApproved    1593 non-null object
HRA                     1591 non-null object
LastUpdated             1593 non-null object
OBJECTID                1523 non-null float64
PPNO                    1523 non-null float64
PARCEL_ACRES            1523 non-null float64
PARCEL_SQFT             1523 non-null float64
JURISDICTION            1523 non-null object
SHAPE                   1523 non-null geometry
dtypes: float64(5), geometry(1), int64(2), object(9)
memory usage: 211.7+ KB


In [ ]:
# start timer
startTimer = datetime.now()

# # copy parcel master to local gdb
# # parcel base
# parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Base"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelsBase")

# # create spatial dataframe from parcel feature class
# sdfParcels = pd.DataFrame.spatial.from_featureclass('ParcelsBase')
# sdfParcels.info()


# # load pickle from dataframe
# with open(os.path.join(dfDump , "dfIPES.pickle"), 'rb') as f:
#     dfIPES = pickle.load(f)

dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

# print data frame 
dfIPES.info()

# filter for active and official
dfIPES_merge = dfIPES[(dfIPES.Status == "Active") 
                            & (dfIPES.IPESScoreType == "Official")]

data_frames = [dfIPES_merge, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'], how='left'), data_frames)

# print dataframe info
df.info()

# export merged data frame to staging workspace
df.spatial.to_featureclass(location= workspace + "\Parcel_IPES")

# input fc
inputFC = "Parcel_IPES"

print("Imported arcpy, os, csv, and datatime modules.\n")
print(arcpy.ProductInfo())
## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = "Parcel_IPES"

# Change this to the path of your output FC
outfc = sdeCollect + "\\sde_collection.SDE.Parcel\\sde_collection.SDE.Parcel_LTinfo_IPES"

# feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"

fieldnames =  ['APN','PPNO','Status','ParcelNickname','IPESScore','IPESScoreType',
               'BaseAllowableCoveragePercent', 'IPESTotalAllowableCoverageSqFt',
               'ParcelHasDOAC', 'HistoricOrImportedIpesScore', 'CalculationDate',
               'FieldEvaluationDate', 'RelativeErosionHazardScore', 'RunoffPotentialScore',
               'AccessScore', 'UtilityInSEZScore', 'ConditionOfWatershedScore',
               'AbilityToRevegetateScore', 'WaterQualityImprovementsScore',
               'ProximityToLakeScore', 'LimitedIncentivePoints','TotalParcelArea',
               'IPESBuildingSiteArea', 'SEZLandArea','SEZSetbackArea','InternalNotes',
               'PublicNotes','PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE@']

#--------------------------------------------------------------------------------------------------------#

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print ("Unregistering feature dataset as versioned...")
# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("Finished unregistering feature dataset as versioned.")
# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeCollect, "ALL")

print("\nRegistering feature dataset as versioned...")
# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
print("\nFinished registering feature dataset as versioned.")

# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))

In [ ]:
# in memory output file path
wk_memory = "in_memory" + "\\"

# network path to connection files
filePath = "C:\\GIS\\DBConnections"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollectSDE = os.path.join(filePath, "Collection_SDE.sde")
sdeCollectBMP = os.path.join(filePath, "Collection_BMP.sde")
bmpDB = os.path.join(filePath, "BMP.sde")

# BMP Land use table
bmpLU =  bmpDB + "\\tahoebmpsde.dbo.v_BMPStatus"

# parcel feature classes to get updated
parcelMaster = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
parcelSimple = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcels_Simplified"

# feature dataset path
landuseFD = "sde_collection.SDE.LandUse"

# feature dataset variable using db owner credentials to unregister and register versions
featureDataset = os.path.join(sdeCollectSDE, landuseFD)

# path to feature class using "BMP_UPDATE" db credentials
parcelCollect = os.path.join(sdeCollectBMP, landuseFD, "sde_collection.SDE.Parcel_Collection")

# function to transfer data
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList, sqlWhere):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList, where_clause = sqlWhere)}  
    print ("transfering...")
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# start an edit session to apply changes to Parcel Master
edit = arcpy.da.Editor(sdeCollectSDE)
print ("edit created")

try:
    # unregister feature dataset as versioned using SDE credentials
    arcpy.UnregisterAsVersioned_management(featureDataset,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print("feature dataset unregistered")
    # start an edit session
    edit.startEditing()
    print("edit started")
    edit.startOperation()
    print("operation started")
    # Perform edits
    ## transfer bmp land use values to parcel collect
    fieldJoinCalc(parcelCollect, ['APN', 'TRPA_LANDUSE_DESCRIPTION'], bmpLU, ['APN', 'LANDUSE'], "BMPStatus = 'BMP'")
    print("The 'TRPA_LANDUSE_DESCRIPTION' field in the parcel collection data has been updated")
    # stop edits
    edit.stopOperation()
    print("operation stopped")
    edit.stopEditing(True)  ## Stop the edit session with True to save the changes
    print("edit stopped")
    # register feature dataset as versioned using SDE credentials
    arcpy.RegisterAsVersioned_management(featureDataset, "EDITS_TO_BASE")
except Exception as err:
    print(err)
    if edit.isEditing:
        edit.stopOperation()
        print("operation stopped in except")
        edit.stopEditing(False)  ## Stop the edit session with False to abandon the changes
        print("edit stopped in except")
finally:
    # Cleanup
    arcpy.ClearWorkspaceCache_management()


# Identify Old APNs

## Parcel Tracker

In [12]:
# merge parcel_master and ipes
dfLTInfo = pd.merge(sdfParcels, dfLTAPN, on='APN', how='outer')
df = dfLTInfo[(dfLTInfo.OBJECTID.isnull())]
dfNull = df[['APN', 'Status']]
dfNull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9697 entries, 61263 to 70959
Data columns (total 2 columns):
APN       9697 non-null object
Status    9697 non-null object
dtypes: object(2)
memory usage: 227.3+ KB


In [13]:
# merge parcel_master and ipes
dfLTInfo = pd.merge(sdfParcels, dfLTAPN, on='APN', how='outer')
df = dfLTInfo[(dfLTInfo.OBJECTID.isnull()) & (dfLTInfo.Status == 'Inactive')]
dfNull = df[['APN', 'Status']]
dfNull

,APN,Status
61263,001-010-02,Inactive
61264,001-010-03,Inactive
61265,001-010-04,Inactive
61266,001-010-05,Inactive
61267,001-010-07,Inactive
61268,001-010-11,Inactive
61269,001-010-13,Inactive
61270,001-020-01,Inactive
61271,001-020-02,Inactive
61272,001-020-03,Inactive


## LCV

In [19]:
# merge parcel_master and ipes
dfLCVNULL = pd.merge(sdfParcels, dfLCV, on='APN', how='outer')
df = dfLCVNULL[(dfLCVNULL.OBJECTID.isnull() & (dfLCVNULL.Status == 'Inactive'))]
dfNull = df[['APN', 'Status']]
dfNull

,APN,Status
61263,001-050-09,Inactive
61264,005-113-04,Inactive
61265,005-113-05,Inactive
61266,005-331-08,Inactive
61267,014-274-10,Inactive
61268,014-274-11,Inactive
61269,014-274-12,Inactive
61270,014-291-02,Inactive
61271,014-360-09,Inactive
61272,015-103-27,Inactive


In [24]:
dfLCVNULL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62229 entries, 0 to 62228
Data columns (total 90 columns):
OBJECTID                          61263 non-null float64
APN                               62229 non-null object
PPNO                              61263 non-null float64
HSE_NUMBR                         61263 non-null float64
UNIT_NUMBR                        61263 non-null object
STR_DIR                           61263 non-null object
STR_NAME                          61263 non-null object
STR_SUFFIX                        61263 non-null object
APO_ADDRESS                       61263 non-null object
PSTL_TOWN                         61263 non-null object
PSTL_STATE                        61263 non-null object
PSTL_ZIP5                         61263 non-null object
OWN_FIRST                         61263 non-null object
OWN_LAST                          61263 non-null object
OWN_FULL                          61263 non-null object
MAIL_ADD1                         61263 non-null

In [23]:
dfLCVNULL

,OBJECTID,APN,PPNO,HSE_NUMBR,UNIT_NUMBR,STR_DIR,STR_NAME,STR_SUFFIX,APO_ADDRESS,PSTL_TOWN,...,DeterminationDate,EstimatedOrVerified,IPESPresent,IPESSqFt,ParcelNickname,SitePlanUrl,Status,TotalAreaSqFt,UpdatedBy,UpdatedOn
0,4.0,048-042-03,4804203.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,02/23/2006 12:00 AM,Verified,No,NaN,None,https://qa.laketahoeinfo.org/FileResource/Disp...,Active,4745879.0,Adele Gerz,08/09/2016 11:56 AM
1,5.0,048-140-01,4814001.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,048-140-02,4814002.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.0,048-140-03,4814003.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,02/23/2006 12:00 AM,Verified,No,NaN,None,https://qa.laketahoeinfo.org/FileResource/Disp...,Active,217843.0,Adele Gerz,08/09/2016 12:07 PM
4,8.0,048-140-04,4814004.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,02/27/2006 12:00 AM,Verified,No,NaN,None,https://qa.laketahoeinfo.org/FileResource/Disp...,Active,28126586.0,Adele Gerz,08/09/2016 12:20 PM
5,9.0,055-010-07,5501007.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10.0,055-010-08,5501008.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14.0,055-010-26,5501026.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15.0,122-051-01,12205101.0,643.0,,,2ND CREEK,DR,643 2ND CREEK DR,Incline Village,...,10/02/1989 12:00 AM,Verified,No,NaN,None,None,Active,28132.0,Sean Tevlin,06/30/2016 9:40 AM
9,16.0,122-051-02,12205102.0,640.0,,,WOODRIDGE,CIR,640 WOODRIDGE CIR,Incline Village,...,08/11/1992 12:00 AM,Verified,No,NaN,None,None,Active,27868.0,Sean Tevlin,06/30/2016 9:41 AM


## IPES

In [27]:
# merge parcel_master and ipes
dfIPESNULL = pd.merge(sdfParcels, dfIPES, on='APN', how='outer')
df = dfIPESNULL[(dfIPESNULL.OBJECTID.isnull())]
dfNull = df[['APN', "JURISDICTION", 'IPESScore', 'Status']]
dfNull

,APN,JURISDICTION,IPESScore,Status
63192,001-010-05,NaN,815.0,Inactive
63193,001-010-07,NaN,673.0,Inactive
63194,001-010-07,NaN,718.0,Inactive
63195,001-020-03,NaN,699.0,Inactive
63196,001-020-03,NaN,650.0,Inactive
63197,001-020-16,NaN,725.0,Inactive
63198,001-030-08,NaN,851.0,Inactive
63199,001-030-19,NaN,791.0,Inactive
63200,001-030-22,NaN,652.0,Inactive
63201,001-040-02,NaN,880.0,Inactive


In [20]:
dfIPESNULL

,OBJECTID,APN,PPNO,HSE_NUMBR,UNIT_NUMBR,STR_DIR,STR_NAME,STR_SUFFIX,APO_ADDRESS,PSTL_TOWN,...,PublicNotes,RelativeErosionHazardScore,RunoffPotentialScore,SEZLandArea,SEZSetbackArea,ScoreSheetUrl,Status,TotalParcelArea,UtilityInSEZScore,WaterQualityImprovementsScore
0,4.0,048-042-03,4804203.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,048-140-01,4814001.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,048-140-02,4814002.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.0,048-140-03,4814003.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.0,048-140-04,4814004.0,0.0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9.0,055-010-07,5501007.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10.0,055-010-08,5501008.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14.0,055-010-26,5501026.0,0.0,,,US HIGHWAY 395 S,,0 US HIGHWAY 395 S,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15.0,122-051-01,12205101.0,643.0,,,2ND CREEK,DR,643 2ND CREEK DR,Incline Village,...,,326.0,98.0,0.0,0.0,https://qa-parcels.laketahoeinfo.org/IpesScore...,Active,NaN,110.0,34.0
9,16.0,122-051-02,12205102.0,640.0,,,WOODRIDGE,CIR,640 WOODRIDGE CIR,Incline Village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# merge parcel_master and ipes
dfParcelIPES = pd.merge(df1, df2, on='APN', how='outer')
# merge merged retired list
dfParcelIPESRetired = pd.merge(dfParcelIPES, df3, on='APN', how='outer')
# merge zillow listing
dfParcelDevelopment =  pd.merge(dfParcelIPESRetired, df4, on='Address', how='outer')
# view data
dfParcelDevelopment.head()

In [ ]:
dfnotRetired = dfParcelDevelopment.SOURCE.notna()
dfnotRetired.head()

In [ ]:
dfParcelDevelopment.count()

In [ ]:
dfParcelDevelopment.groupby('TRPA_LANDUSE_DESCRIPTION')['APN'].nunique()

## Exploratory Data Analysis

# Send Email

In [ ]:
# email packages
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

## Data Frames to CSV

In [3]:
dfLCV.to_csv(r"C:\GIS\PROJECT\LTInfo\Data\dfLCV.csv")

In [25]:
emailfrom = "mbindl@trpa.org"
emailto = "mbindl@trpa.org"
dataframe = "dfLCV.csv"
fileToSend = ('C:\\GIS\\PROJECT\\LTInfo\\Data\\' + dataframe)
# username = "username"
# password = "password"

msg = MIMEMultipart()
msg["From"] = emailfrom
msg["To"] = emailto
msg["Subject"] = "CSV of Missing APNs"
msg.preamble = "This is a test"

ctype, encoding = mimetypes.guess_type(fileToSend)
if ctype is None or encoding is not None:
    ctype = "application/octet-stream"

maintype, subtype = ctype.split("/", 1)

if maintype == "text":
    fp = open(fileToSend)
    # Note: we should handle calculating the charset
    attachment = MIMEText(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "image":
    fp = open(fileToSend, "rb")
    attachment = MIMEImage(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "audio":
    fp = open(fileToSend, "rb")
    attachment = MIMEAudio(fp.read(), _subtype=subtype)
    fp.close()
else:
    fp = open(fileToSend, "rb")
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
attachment.add_header("Content-Disposition", "attachment", filename="APN_Changes.csv")
msg.attach(attachment)

server = smtplib.SMTP("mail.smtp2go.com", 25)
server.starttls()
# server.login(username,password)
server.sendmail(emailfrom, emailto, msg.as_string())
server.quit()

SMTPRecipientsRefused: {'mbindl@trpa.org': (550, b'relay access denied - please authenticate')}